# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.233705775441                   -0.50    8.0
  2   -7.250039791853       -1.79       -1.39    1.0
  3   -7.251240607694       -2.92       -1.94    4.0
  4   -7.251247834670       -5.14       -2.09    4.0
  5   -7.251324633212       -4.11       -2.58    3.0
  6   -7.251334513825       -5.01       -2.83    2.0
  7   -7.251338772824       -5.37       -4.10    1.0
  8   -7.251338793683       -7.68       -4.24    4.0
  9   -7.251338798305       -8.34       -4.79    3.0
 10   -7.251338798644       -9.47       -5.22    1.0
 11   -7.251338798704      -10.22       -6.01    2.0
 12   -7.251338798704      -12.27       -6.48    3.0
 13   -7.251338798704      -13.30       -6.74    3.0
 14   -7.251338798705      -13.35       -7.68    2.0
 15   -7.251338798705   +  -14.75       -7.86    4.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.05465931723913529
[ Info: Arnoldi iteration step 2: normres = 0.5555722388160905
[ Info: Arnoldi iteration step 3: normres = 0.9088454432883659
[ Info: Arnoldi iteration step 4: normres = 0.2996380390666419
[ Info: Arnoldi iteration step 5: normres = 0.4258747602696918
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.39e-02, 6.15e-02, 3.10e-01, 2.81e-01, 3.35e-02)
[ Info: Arnoldi iteration step 6: normres = 0.3713177730664047
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (1.13e-02, 2.26e-01, 2.66e-01, 1.02e-01, 6.86e-02)
[ Info: Arnoldi iteration step 7: normres = 0.06349116643945996
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (3.58e-04, 1.58e-02, 8.53e-03, 2.44e-02, 4.78e-02)
[ Info: Arnoldi iteration step 8: normres = 0.10091271946604723
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (1.55e-0